In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from IPython.display import display
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Statistics & Mathematics
import scipy.stats as stats
import math

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Preprocessing data
from sklearn.preprocessing import RobustScaler, StandardScaler

# Model Selection for Cross Validation
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

# Machine Learning metrics
from sklearn.metrics import mean_squared_error, r2_score

# ML algorithms
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, StackingRegressor, AdaBoostRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.cluster import KMeans
import optuna

# Encoder of categorical variables
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Hiding warnings 
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("D:\\python project\\salaries\\Latest_Data_Science_Salaries.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3300 entries, 0 to 3299
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job Title           3300 non-null   object
 1   Employment Type     3300 non-null   object
 2   Experience Level    3300 non-null   object
 3   Expertise Level     3300 non-null   object
 4   Salary              3300 non-null   int64 
 5   Salary Currency     3300 non-null   object
 6   Company Location    3300 non-null   object
 7   Salary in USD       3300 non-null   int64 
 8   Employee Residence  3300 non-null   object
 9   Company Size        3300 non-null   object
 10  Year                3300 non-null   int64 
dtypes: int64(3), object(8)
memory usage: 283.7+ KB


In [3]:
df.head()

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
0,Data Engineer,Full-Time,Senior,Expert,210000,United States Dollar,United States,210000,United States,Medium,2023
1,Data Engineer,Full-Time,Senior,Expert,165000,United States Dollar,United States,165000,United States,Medium,2023
2,Data Engineer,Full-Time,Senior,Expert,185900,United States Dollar,United States,185900,United States,Medium,2023
3,Data Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2023
4,Data Scientist,Full-Time,Senior,Expert,140000,United States Dollar,United States,140000,United States,Medium,2023


In [4]:
def dataframe_description(df):
    """
    This function prints some basic info on the dataset.
    """
    categorical_features = []
    continuous_features = []
    binary_features = []
    
    for col in df.columns:
        if df[col].dtype == object:
            categorical_features.append(col)
        else:
            if df[col].nunique() <= 2:
                binary_features.append(col)
            else:
                continuous_features.append(col)
    
    print("\n{} shape: {}".format(type(df).__name__, df.shape))
    print("\n{:,.0f} samples".format(df.shape[0]))
    print("\n{:,.0f} attributes".format(df.shape[1]))
    print(f'\nMissing Data: \n')
    print(df.isnull().sum())
    print(f'\nDuplicates: {df.duplicated().sum()}')
    print(f'\nData types: \n')
    print(df.dtypes)
    print(f'\nCategorical features: \n')
    if len(categorical_features) == 0:
        print('No Categorical Features')
    else:
        for feature in categorical_features:
            print(feature)
    print(f'\nContinuous features: \n')
    if len(continuous_features) == 0:
        print('No Continuous Features')
    else:
        for feature in continuous_features:
            print(feature)
    print(f'\nBinary features: \n')
    if len(binary_features) == 0:
        print('No Binary Features')
    else:
        for feature in binary_features:
            print(feature)
    print(f'\n{type(df).__name__} Head: \n')
    display(df.head(5))
    print(f'\n{type(df).__name__} Tail: \n')
    display(df.tail(5))

In [5]:
def plot_correlation(df):
    '''
    This function is resposible to plot a correlation map among features in the dataset
    '''
    corr = np.round(df.corr(), 2)
    mask = np.triu(np.ones_like(corr, dtype = bool))
    c_mask = np.where(~mask, corr, 100)

    c = []
    for i in c_mask.tolist()[1:]:
        c.append([x for x in i if x != 100])
    
    fig = ff.create_annotated_heatmap(z=c[::-1],
                                      x=corr.index.tolist()[:-1],
                                      y=corr.columns.tolist()[1:][::-1],
                                      colorscale = 'bluyl')

    fig.update_layout(title = {'text': '<b>Feature Correlation <br> <sup>Heatmap</sup></b>'},
                      height = 650, width = 650,
                      margin = dict(t=150, l = 80),
                      template = 'simple_white',
                      yaxis = dict(autorange = 'reversed'))

    fig.add_trace(go.Heatmap(z = c[::-1],
                             colorscale = 'bluyl',
                             showscale = True,
                             visible = False))
    fig.data[1].visible = True

    fig.show()

In [6]:
def plot_distplot(df, x):  
    '''
    This function creates a distribution plot for continuous variables
    '''
    
    feature = df[x]

    fig = ff.create_distplot([feature], [x], show_hist=False)

    fig.update_layout(
        title={'text': f'<b>Distplot <br> <sup>{x}</sup></b>',
               'xanchor': 'left',
               'x': 0.05},
        height=600,
        width=1000,
        margin=dict(t=100),
        template='plotly_white',
        showlegend=True
    )

    fig.show()

In [7]:
def plot_histogram_matrix(df):
    
    '''
    This function identifies all continuous features within the dataset and plots
    a matrix of histograms for each attribute
    '''
    
    continuous_features = []
    for feat in df.columns:
        if df[feat].nunique() > 2:
            continuous_features.append(feat)
    num_cols = 2
    num_rows = (len(continuous_features) + 1) // num_cols

    fig = make_subplots(rows=num_rows, cols=num_cols)

    for i, feature in enumerate(continuous_features):
        row = i // num_cols + 1
        col = i % num_cols + 1

        fig.add_trace(
            go.Histogram(
                x=df[feature],
                name=feature
            ),
            row=row,
            col=col
        )

        fig.update_xaxes(title_text=feature, row=row, col=col)
        fig.update_yaxes(title_text='Frequency', row=row, col=col)
        fig.update_layout(
            title=f'<b>Histogram Matrix<br> <sup> Continuous Features</sup></b>',
            showlegend=False
        )

    fig.update_layout(
        height=350 * num_rows,
        width=1000,
        margin=dict(t=100, l=80),
        template='plotly_white'
    )

    fig.show()

In [8]:
def plot_histogram_matrix(df):
    
    '''
    This function identifies all continuous features within the dataset and plots
    a matrix of histograms for each attribute
    '''
    
    continuous_features = []
    for feat in df.columns:
        if df[feat].nunique() > 2:
            continuous_features.append(feat)
    num_cols = 2
    num_rows = (len(continuous_features) + 1) // num_cols

    fig = make_subplots(rows=num_rows, cols=num_cols)

    for i, feature in enumerate(continuous_features):
        row = i // num_cols + 1
        col = i % num_cols + 1

        fig.add_trace(
            go.Histogram(
                x=df[feature],
                name=feature
            ),
            row=row,
            col=col
        )

        fig.update_xaxes(title_text=feature, row=row, col=col)
        fig.update_yaxes(title_text='Frequency', row=row, col=col)
        fig.update_layout(
            title=f'<b>Histogram Matrix<br> <sup> Continuous Features</sup></b>',
            showlegend=False
        )

    fig.update_layout(
        height=350 * num_rows,
        width=1000,
        margin=dict(t=100, l=80),
        template='plotly_white'
    )

    fig.show()

In [9]:
def scatterplot(df, x, y):
    '''
    This function takes a dataframe and X and y axes to plot a scatterplot
    '''

    color_dict = {
        0: 'orange',
        1: 'blue',
        2: 'green',
        3: 'red',
        4: 'black'
    }

    # Randomly select a color index from the dictionary
    color_index = random.choice(list(color_dict.keys()))
    color = color_dict[color_index]

    fig = px.scatter(df, y=y, x=x)
    fig.update_traces(marker=dict(size=10, color=color))
    fig.update_layout(
        title={'text': f'<b>Scatterplot <br> <sup>{x} x {y}</sup></b>'},
        height=750,
        width=850,
        margin=dict(t=100, l=80),
        template='simple_white'
    )
    fig.show()

In [10]:
def plot_clustered_scatterplot(df, y, x):
    '''
    This function takes a dataframe, x, and y axes to plot a scatterplot colored accordingly to clusters
    It also prints a count of values for each cluster
    '''
    fig = px.scatter(df,
                     y = y,
                     x = x,
                     color = 'Cluster', symbol = 'Cluster')

    fig.update_traces(marker = dict(size = 10))

    fig.update(layout_coloraxis_showscale=False)

    fig.update_layout(title = {'text': f'<b>Clustered Scatterplot <br> <sup> {y} x {x} </sup></b>',
                              'xanchor': 'left',
                              'x': 0.05},
                     height = 750, width = 1000,
                     margin = dict(t=100),
                     template = 'seaborn',
                     showlegend = True)

    fig.show()

    print('Cluster Count:')
    print(f'{X.Cluster.value_counts()}')
    

In [11]:
def top15_barplot(df, feat):    
    
    '''
    This function is supposed to organize the 15 top value counts of any attribute and plot a Barplot
    '''
    
    top15 = df[feat].value_counts()[:15]
    fig = px.bar(y=top15.values, 
                 x=top15.index, 
                 color = top15.index,
                 text=top15.values)

    fig.update_layout(title=f'<b>Top 15 most frequent {feat}<br> <sup> Barplot</sup></b>',
                      xaxis=dict(title=f'{feat}'),
                      yaxis=dict(title='Count'),
                      legend=dict(title=f'{feat}'),
                      showlegend=True,
                      height=600,
                      width=1000,
                      margin=dict(t=100, l=80),
                      template='plotly_white')
    fig.show()

In [12]:
def shapiro_wilk_test(df):
    '''
    This function performs a Shapiro-Wilk test to check if the data is normally distributed or not, as well as skewness
    '''
    print(f'\033[1mShapiro-Wilk Test & Skewness:\033[0m')
    print('\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  \n')

    numeric_columns = df.select_dtypes(include=['float', 'int']).columns

    for feature in numeric_columns:
        stats, p_value = shapiro(df[feature])

        if p_value < 0.05:
            text = f'{feature} Does Not Seem to be Normally Distributed'
        else:
            text = f'{feature} Seems to be Normally Distributed'

        print(f'{feature}')
        print(f'\n  Shapiro-Wilk Statistic: {stats:.2f}')
        print(f'\n  Shapiro-Wilk P-value: {p_value}')
        print(f'\n  Skewness: {np.round(skew(df[feature]), 2)}')
        print(f'\n  Conclusion: {text}')
        print('\n===============================================================================================')

    print('\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  \n')
    print(f'\033[1mEnd of Shapiro-Wilk Test\033[0m')

In [13]:
def individual_boxplot(df, y, x):    
    
    '''
    This function plots a Y and X boxplot
    
    '''
    fig = px.box(df, y= y , x = x, color= x)

    fig.update_layout(title=f'<b>Boxplot<br> <sup> {y} by {x}</sup></b>',
                      showlegend=False,
                      yaxis=dict(tickangle= -45),
                      height=600,
                      width=1000,
                      margin=dict(t=100, l=80),
                      template='plotly_white')

    fig.show()

In [14]:
def plot_barplot(df, x):    
    
    '''
    This function plots different barplots by taking a dataframe and a column
    '''
    
    new_df = df[x].value_counts()
    fig = px.bar(y=new_df.values, 
                 x=new_df.index, 
                 color = new_df.index,
                 text=new_df.values)

    fig.update_layout(title=f'<b>{x}<br> <sup> Among high-earning outliers</sup></b>',
                      xaxis=dict(title=f'{x}'),
                      yaxis=dict(title='Count'),
                      legend=dict(title=f'{x}'),
                      showlegend=True,
                      height=650, 
                      width=1000,
                      margin=dict(t=100, l=80),
                      template='plotly_white')
    fig.show()

In [15]:
continuous_features = ['Salary', 'Salary in USD', 'Year']
plot_histogram_matrix(df[continuous_features])

In [60]:
continuous_features = ['Salary', 'Salary in USD', 'Year']
plot_histogram_matrix(df[continuous_features])

In [18]:
plot_distplot(df, 'Salary in USD')

In [19]:
categorical_columns = ['Employment Type', 'Experience Level', 'Expertise Level','Company Size', 'Year']
for feat in categorical_columns:
    individual_boxplot(df, 'Salary in USD', feat)

In [20]:
df.query("`Salary in USD` == 416000") # Checking for the 416k/year outlier

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
3228,Principal Data Scientist,Contract,Executive,Director,416000,United States Dollar,United States,416000,United States,Small,2021


In [21]:
df.query("`Salary in USD` == 100000 and `Employment Type` == 'Freelance'") # Checking for the high-earning freelance

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
2337,Machine Learning Engineer,Freelance,Entry,Junior,100000,United States Dollar,"Iran, Islamic Republic of",100000,"Iran, Islamic Republic of",Medium,2022
3044,Data Scientist,Freelance,Mid,Intermediate,100000,United States Dollar,United States,100000,Canada,Medium,2022


In [ ]:
top15_barplot(df, 'Job Title')

In [ ]:
top_job_titles = df['Job Title'].value_counts(ascending=False).head(10).index

fig = go.Figure()

for feat in top_job_titles:
    filtered_df = df[df['Job Title'] == feat]
    box_plot = go.Box(y=filtered_df['Salary in USD'], name=feat)
    fig.add_trace(box_plot)


fig.update_layout(
    title='<b>Who is making more money? <br> <sup> Among the most frequent job titles</sup></b>',
    showlegend=True,
    legend_title_text='Job Title',
    yaxis=dict(title='Salary in USD'),
    xaxis=dict(title=''),
    height=600,
    width=1000,
    margin=dict(t=100, l=80),
    template='plotly_white'
)

fig.show()

In [ ]:
highest_earning_titles = df.groupby('Job Title')['Salary in USD'].median().nlargest(10).index

median_salaries = []
std_devs = []

for feat in highest_earning_titles:
    filtered_df = df[df['Job Title'] == feat]
    median_salaries.append(filtered_df['Salary in USD'].median())
    std_devs.append(filtered_df['Salary in USD'].std())

fig = go.Figure(data=go.Bar(
    x=highest_earning_titles,
    y=median_salaries,
    error_y=dict(
        type='data',
        array=std_devs,
        visible=True
    )
))

fig.update_layout(
    title='<b>Who is making more money? <br> <sup> Unfiltered highest-earning job titles</sup></b>',
    yaxis=dict(title='Salary in USD'),
    xaxis=dict(title='Job Title'),
    height=600,
    width=1000,
    margin=dict(t=100, l=80),
    template='plotly_white'
)

fig.show()

In [22]:
df.query("`Job Title` == 'Analytics Engineering Manager'") # Filtering people with Analytics Engineering Manager Job Title

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
826,Analytics Engineering Manager,Full-Time,Senior,Expert,325000,British Pound Sterling,United Kingdom,399880,United Kingdom,Large,2023


In [23]:
min_people_threshold = 5

job_title_counts = df['Job Title'].value_counts()
highest_earning_titles = job_title_counts[job_title_counts >= min_people_threshold].head(10).index

median_salaries = []
std_devs = []

for feat in highest_earning_titles:
    filtered_df = df[df['Job Title'] == feat]
    median_salaries.append(filtered_df['Salary in USD'].median())
    std_devs.append(filtered_df['Salary in USD'].std())

sorted_titles, sorted_salaries, sorted_std_devs = zip(*sorted(zip(highest_earning_titles, median_salaries, std_devs), key=lambda x: x[1], reverse=True))
    
fig = go.Figure(data=go.Bar(
    x=sorted_titles,
    y=sorted_salaries,
    error_y=dict(
        type='data',
        array=sorted_std_devs,
        visible=True
    )
))

fig.update_layout(
    title='<b>Who is making more money? <br> <sup> Filtered highest-earning job titles </sup></b>',
    yaxis=dict(title='Salary in USD'),
    xaxis=dict(title='Job Title'),
    height=600,
    width=1000,
    margin=dict(t=100, l=80),
    template='plotly_white'
)

fig.show()

In [24]:
top5_job_titles = df['Job Title'].value_counts().head(5).index
filtered_df = df[df['Job Title'].isin(top5_job_titles)]
counts = filtered_df.groupby(['Year', 'Job Title']).size().reset_index(name='Count')

engineer_counts = counts[counts['Job Title'] == 'Data Engineer']
scientist_counts = counts[counts['Job Title'] == 'Data Scientist']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=engineer_counts['Year'],
    y=engineer_counts['Count'],
    mode='lines',
    name='Data Engineer'
))

fig.add_trace(go.Scatter(
    x=scientist_counts['Year'],
    y=scientist_counts['Count'],
    mode='lines',
    name='Data Scientist'
))

fig.update_layout(title=f'<b>Data Engineers vs Data Scientists<br> <sup> Evolution through the years - line plot</sup></b>',
                  showlegend=True,
                  xaxis=dict(
                      title='Year',
                      tickmode='linear',
                      tick0=2020,
                      dtick=1
                  ),
                  yaxis_title='Count',
                  height=700,
                  width=1000,
                  margin=dict(t=250, l=80),
                  template='plotly_white')

fig.show()

In [25]:
engineer_counts = counts[counts['Job Title'] == 'Data Engineer']
scientist_counts = counts[counts['Job Title'] == 'Data Scientist']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=engineer_counts['Year'],
    y=engineer_counts['Count'],
    name='Data Engineer'
))

fig.add_trace(go.Bar(
    x=scientist_counts['Year'],
    y=scientist_counts['Count'],
    name='Data Scientist'
))

fig.update_layout(
    title='<b>Data Engineers vs Data Scientists<br><sup> Evolution through the years - Barplot</sup></b>',
    showlegend=True,
    xaxis=dict(
        title='Year',
        tickmode='linear',
        tick0=2020,
        dtick=1
    ),
    yaxis=dict(title='Count'),
    height=700,
    width=1000,
    margin=dict(t=250, l=80),
    template='plotly_white'
)

fig.show()

In [26]:
grouped_df = df.groupby(['Year', 'Employee Residence']).size().reset_index(name='Count')
top5_per_year = grouped_df.groupby('Year').apply(lambda x: x.nlargest(5, 'Count')).reset_index(drop=True)

fig = px.bar(top5_per_year, x='Year', y='Count', color='Employee Residence', barmode='group')

fig.update_layout(title=f'<b>Most Frequent Countries of Employee Residence<br> <sup> By year</sup></b>',
                      xaxis=dict(title=f'Year'),
                      yaxis=dict(title='Count (Log Scale)', type = 'log'),
                      legend=dict(title=f'Employee Residence'),
                      showlegend=True,
                      height=600,
                      width=1000,
                      margin=dict(t=100, l=80),
                      template='plotly_white')

fig.show()

In [27]:
residence_counts = df['Employee Residence'].value_counts().reset_index()
residence_counts.columns = ['Employee Residence', 'Count']

fig = go.Figure(data=go.Choropleth(
    locations=residence_counts['Employee Residence'],
    locationmode='country names',
    z=residence_counts['Count'],
    colorscale='deep',
    reversescale=True,
    colorbar=dict(title='Count'),
))

fig.update_layout(title=f'<b>Total Counts of Employee Residence<br> <sub>Data professionals around the world</sub></b>',
                  geo=dict(showframe=False, showcoastlines=True, projection_type='natural earth'),
                  height=800,
                  margin=dict(t=100, l=80),
                  template='plotly_white')

fig.show()

In [28]:
location_counts = df['Company Location'].value_counts().reset_index()
location_counts.columns = ['Company Location', 'Count']

fig = go.Figure(data=go.Choropleth(
    locations=location_counts['Company Location'],
    locationmode='country names',
    z=location_counts['Count'],
    colorscale='deep',
    reversescale=True,
    colorbar=dict(title='Count'),
))

fig.update_layout(title=f'<b>Total Counts of Company Location<br> <sub>Who is hiring data professionals?</sub></b>',
                  geo=dict(showframe=False, showcoastlines=True, projection_type='natural earth'),
                  height=800,
                  margin=dict(t=100, l=80),
                  template='plotly_white')

fig.show()

In [29]:
fig = px.box(df, x = 'Salary in USD')

fig.update_layout(title=f'<b>Boxplot<br> <sup> Salary in USD</sup></b>',
                  showlegend=False,
                  yaxis=dict(tickangle= -45),
                  height=400,
                  width=1000,
                  margin=dict(t=100, l=80),
                  template='plotly_white')

fig.show()

In [30]:
outliers = df.query("`Salary in USD` >= 350000") # Filtering high earners
outliers.head(10) # Displaying Dataframe 

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
304,ML Engineer,Full-Time,Senior,Expert,383910,United States Dollar,United States,383910,United States,Medium,2023
431,Machine Learning Engineer,Full-Time,Senior,Expert,392000,United States Dollar,United States,392000,United States,Medium,2023
457,Data Scientist,Full-Time,Senior,Expert,359170,United States Dollar,United States,359170,United States,Medium,2023
494,ML Engineer,Full-Time,Senior,Expert,365630,United States Dollar,United States,365630,United States,Medium,2023
647,Research Engineer,Full-Time,Senior,Expert,370000,United States Dollar,United States,370000,United States,Medium,2023
679,Research Scientist,Full-Time,Senior,Expert,370000,United States Dollar,United States,370000,United States,Medium,2023
707,Research Scientist,Full-Time,Senior,Expert,374000,United States Dollar,United States,374000,United States,Medium,2023
826,Analytics Engineering Manager,Full-Time,Senior,Expert,325000,British Pound Sterling,United Kingdom,399880,United Kingdom,Large,2023
1242,Analytics Engineer,Full-Time,Mid,Intermediate,350000,British Pound Sterling,United Kingdom,430640,United Kingdom,Medium,2023
1469,Director of Data Science,Full-Time,Executive,Director,353200,United States Dollar,United States,353200,United States,Medium,2023


In [31]:
job_titles = outliers['Job Title'].value_counts()
fig = px.bar(y=job_titles.values, 
             x=job_titles.index, 
             color = job_titles.index,
             text=job_titles.values)

fig.update_layout(title=f'<b>Job Titles Frequence<br> <sup> Among high-earning outliers</sup></b>',
                  xaxis=dict(title=f'Job Titles'),
                  yaxis=dict(title='Count'),
                  legend=dict(title=f'Job Titles'),
                  showlegend=True,
                  height=650, 
                  width=1000,
                  margin=dict(t=100, l=80),
                  template='plotly_white')
fig.show()

In [32]:
residence_counts = outliers['Employee Residence'].value_counts().reset_index()
residence_counts.columns = ['Employee Residence', 'Count']

fig = go.Figure(data=go.Choropleth(
    locations=residence_counts['Employee Residence'],
    locationmode='country names',
    z=residence_counts['Count'],
    colorscale='deep',
    reversescale=True,
    colorbar=dict(title='Count'),
))

fig.update_layout(title=f'<b>Counts of Employee Residence<br> <sub>Among high-earning outliers</sub></b>',
                  geo=dict(showframe=False, showcoastlines=True, projection_type='natural earth'),
                  height=800,
                  margin=dict(t=100, l=80),
                  template='plotly_white')

fig.show()

In [33]:
fig = px.pie(outliers, names = 'Salary Currency', hole = .75)

fig.update_layout(title = {'text': f'<b>Distribution of Salary Currency<br> <sup> Among high-earning outliers</sup></b>'},
                  height = 700, width = 1000,
                  margin = dict(t=250, l = 80),
                  template = 'plotly_white')

fig.show()

In [34]:
cols = ['Employment Type', 'Experience Level', 'Expertise Level', 'Company Size']
for col in cols:
    plot_barplot(outliers, col)

In [35]:
train, test = train_test_split(df, test_size = 0.2, shuffle = True, random_state = 123) # Splitting data

# Printing info on the training set
print(f'\n Train shape: {train.shape}\n')
print(f'\n {len(train)} Samples \n')
print(f'\n {len(train.columns)} Attributes \n')
display(train.head(10))
print('\n' * 2)

# Printing info on the training set
print(f'\n Test shape: {test.shape:}\n')
print(f'\n {len(test)} Samples \n')
print(f'\n {len(test.columns)} Attributes \n')
display(test.head(10))


 Train shape: (2640, 11)


 2640 Samples 


 11 Attributes 



,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
1515,Data Engineer,Full-Time,Mid,Intermediate,151000,United States Dollar,United States,151000,United States,Medium,2023
2066,Data Engineer,Full-Time,Senior,Expert,250000,United States Dollar,United States,250000,United States,Medium,2022
1785,Data Scientist,Full-Time,Senior,Expert,215050,United States Dollar,United States,215050,United States,Medium,2023
117,Data Engineer,Full-Time,Senior,Expert,240500,United States Dollar,United States,240500,United States,Large,2023
1473,Data Scientist,Full-Time,Senior,Expert,297300,United States Dollar,United States,297300,United States,Medium,2023
2413,Data Scientist,Full-Time,Senior,Expert,156600,United States Dollar,United States,156600,United States,Medium,2022
2728,Data Scientist,Full-Time,Mid,Intermediate,85000,Euro,Netherlands,89306,Netherlands,Medium,2022
3270,Business Data Analyst,Full-Time,Entry,Junior,50000,Euro,Luxembourg,59102,Luxembourg,Large,2021
1963,Data Engineer,Full-Time,Executive,Director,196200,United States Dollar,United States,196200,United States,Medium,2023
2135,Data Engineer,Full-Time,Executive,Director,239000,United States Dollar,United States,239000,United States,Medium,2022






 Test shape: (660, 11)


 660 Samples 


 11 Attributes 



,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
1482,MLOps Engineer,Full-Time,Mid,Intermediate,134000,United States Dollar,United States,134000,United States,Medium,2023
2738,Data Analyst,Full-Time,Senior,Expert,117000,United States Dollar,United States,117000,United States,Medium,2022
2743,Machine Learning Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2022
2424,Data Engineer,Full-Time,Senior,Expert,172200,United States Dollar,United States,172200,United States,Medium,2022
805,Machine Learning Engineer,Full-Time,Senior,Expert,180000,United States Dollar,United States,180000,United States,Medium,2023
2534,Big Data Engineer,Full-Time,Senior,Expert,210000,Canadian Dollar,Canada,161311,Canada,Medium,2022
1940,Data Scientist,Full-Time,Executive,Director,300000,United States Dollar,United States,300000,United States,Medium,2023
2876,Data Engineer,Full-Time,Mid,Intermediate,45000,Euro,Greece,47280,Greece,Medium,2022
1756,Data Analyst,Full-Time,Entry,Junior,48000,United States Dollar,United States,48000,United States,Medium,2023
649,Machine Learning Engineer,Full-Time,Senior,Expert,285000,United States Dollar,United States,285000,United States,Medium,2023


In [36]:
train_copy = train.copy() # Copying the training set
train_copy.head(5) # Visualizing

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
1515,Data Engineer,Full-Time,Mid,Intermediate,151000,United States Dollar,United States,151000,United States,Medium,2023
2066,Data Engineer,Full-Time,Senior,Expert,250000,United States Dollar,United States,250000,United States,Medium,2022
1785,Data Scientist,Full-Time,Senior,Expert,215050,United States Dollar,United States,215050,United States,Medium,2023
117,Data Engineer,Full-Time,Senior,Expert,240500,United States Dollar,United States,240500,United States,Large,2023
1473,Data Scientist,Full-Time,Senior,Expert,297300,United States Dollar,United States,297300,United States,Medium,2023


In [37]:
# Defining orders for labels
value_orders = [
    ['Entry', 'Mid', 'Senior', 'Executive'],
    ['Junior', 'Intermediate', 'Expert', 'Director'],
    ['Small', 'Medium', 'Large']
]

# Using OrdinalEncoder to encode 'Experience Level', 'Expertise Level', and 'Company Size' attributes.
oe = OrdinalEncoder(categories = value_orders)
train_copy[['Experience Level', 'Expertise Level', 'Company Size']] = oe.fit_transform(train_copy[['Experience Level', 'Expertise Level', 'Company Size']])
train_copy.head(5)

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
1515,Data Engineer,Full-Time,1.0,1.0,151000,United States Dollar,United States,151000,United States,1.0,2023
2066,Data Engineer,Full-Time,2.0,2.0,250000,United States Dollar,United States,250000,United States,1.0,2022
1785,Data Scientist,Full-Time,2.0,2.0,215050,United States Dollar,United States,215050,United States,1.0,2023
117,Data Engineer,Full-Time,2.0,2.0,240500,United States Dollar,United States,240500,United States,2.0,2023
1473,Data Scientist,Full-Time,2.0,2.0,297300,United States Dollar,United States,297300,United States,1.0,2023


In [38]:
# Selecting columns to be encoded
cols = ['Job Title', 'Employment Type', 'Salary Currency', 'Company Location', 'Employee Residence']

# Encoding
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
encoded_features = pd.DataFrame(ohe.fit_transform(train_copy[cols]), 
                                columns = ohe.get_feature_names_out(cols))
encoded_features.index = train_copy.index
train_copy = train_copy.drop(cols, axis = 1)
train_copy = pd.concat([train_copy, encoded_features], axis = 1)
train_copy

,Experience Level,Expertise Level,Salary,Salary in USD,Company Size,Year,Job Title_AI Architect,Job Title_AI Developer,Job Title_AI Programmer,Job Title_AI Scientist,...,Employee Residence_Sweden,Employee Residence_Switzerland,Employee Residence_Thailand,Employee Residence_Turkey,Employee Residence_Ukraine,Employee Residence_United Arab Emirates,Employee Residence_United Kingdom,Employee Residence_United States,Employee Residence_Uzbekistan,Employee Residence_Viet Nam
1515,1.0,1.0,151000,151000,1.0,2023,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2066,2.0,2.0,250000,250000,1.0,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1785,2.0,2.0,215050,215050,1.0,2023,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
117,2.0,2.0,240500,240500,2.0,2023,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1473,2.0,2.0,297300,297300,1.0,2023,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,1.0,1.0,122500,122500,1.0,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3089,2.0,2.0,144000,144000,2.0,2021,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1766,1.0,1.0,126277,126277,1.0,2023,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1122,0.0,0.0,5500000,41809,2.0,2022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
X, y = train_copy.drop(['Salary in USD', 'Salary'], axis = 1), train_copy['Salary in USD'] # Splitting df into X and y

#Printing info on X and y
print(f'\nX shape: {X.shape}\n')
print(f'\n{len(X)} Samples \n')
print(f'\n{len(X.columns)} Attributes \n')
display(X.head(10))
print('\n')
print(f'\ny shape: {X.shape}\n')
print(f'\n{len(y)} Samples \n')
display(y.head(10))


X shape: (2640, 279)


2640 Samples 


279 Attributes 



,Experience Level,Expertise Level,Company Size,Year,Job Title_AI Architect,Job Title_AI Developer,Job Title_AI Programmer,Job Title_AI Scientist,Job Title_Analytics Engineer,Job Title_Analytics Engineering Manager,...,Employee Residence_Sweden,Employee Residence_Switzerland,Employee Residence_Thailand,Employee Residence_Turkey,Employee Residence_Ukraine,Employee Residence_United Arab Emirates,Employee Residence_United Kingdom,Employee Residence_United States,Employee Residence_Uzbekistan,Employee Residence_Viet Nam
1515,1.0,1.0,1.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2066,2.0,2.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1785,2.0,2.0,1.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
117,2.0,2.0,2.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1473,2.0,2.0,1.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2413,2.0,2.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2728,1.0,1.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3270,0.0,0.0,2.0,2021,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1963,3.0,3.0,1.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2135,3.0,3.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0





y shape: (2640, 279)


2640 Samples 



1515    151000
2066    250000
1785    215050
117     240500
1473    297300
2413    156600
2728     89306
3270     59102
1963    196200
2135    239000
Name: Salary in USD, dtype: int64

In [40]:
cv = KFold(n_splits = 5, shuffle = True, random_state = 123)
cv_splits = list(cv.split(X,y))

In [41]:
regressors = [
    ('CatBoost', CatBoostRegressor(random_state = 123, verbose = False)),
    ('XGboost', XGBRegressor(random_state = 123)),
    ('Ada Boost', AdaBoostRegressor(random_state = 123)),
    ('Histogram-based Gradient Boosting', HistGradientBoostingRegressor(random_state = 123))
]

In [42]:
y_range = y.max() - y.min()  
y_median = y.median()  
y_mean = y.mean() 

print(f"Range of y: {y_range}")
print(f"Median of y: {y_median}")
print(f"Mean of y: {y_mean:.2f}")

Range of y: 435000
Median of y: 136402.5
Mean of y: 142666.50


In [43]:
print('\nCross-Validation:')
for j, (name, clf) in enumerate(regressors):
    scores = []
    r2_scores = []
    
    print('\n')
    print(f'\n{name} Regressor:\n')
    
    for i, (train_index, val_index) in enumerate(cv_splits):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        r2 = r2_score(y_val, y_pred)
        
        n = X_val.shape[0]  
        p = X_val.shape[1]  
        
        adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1)) 
        
        print(f'Fold {i + 1} RMSE = {rmse:.2f}')
        print(f'Fold {i + 1} Adjusted R-squared = {adjusted_r2:.2f}')
        
        scores.append(rmse)
        r2_scores.append(adjusted_r2)
        
        print('===================================================')
    
    if i == len(cv_splits) - 1:
        mean_score = np.mean(scores)
        fold_std = np.std(scores)
        mean_adjusted_r2 = np.mean(r2_scores)  # Compute mean adjusted R-squared
        
        print(f'Mean RMSE = {mean_score:.2f} \u00B1 {fold_std:.3f}')
        print(f'Mean Adjusted R-squared = {mean_adjusted_r2:.2f}')



Cross-Validation:



CatBoost Regressor:

Fold 1 RMSE = 53063.09
Fold 1 Adjusted R-squared = -0.28
Fold 2 RMSE = 53708.96
Fold 2 Adjusted R-squared = -0.25
Fold 3 RMSE = 51754.75
Fold 3 Adjusted R-squared = -0.29
Fold 4 RMSE = 56227.11
Fold 4 Adjusted R-squared = -0.35
Fold 5 RMSE = 54002.19
Fold 5 Adjusted R-squared = -0.34
Mean RMSE = 53751.22 ± 1459.742
Mean Adjusted R-squared = -0.30



XGboost Regressor:

Fold 1 RMSE = 56361.14
Fold 1 Adjusted R-squared = -0.44
Fold 2 RMSE = 54934.81
Fold 2 Adjusted R-squared = -0.31
Fold 3 RMSE = 52863.20
Fold 3 Adjusted R-squared = -0.35
Fold 4 RMSE = 56669.29
Fold 4 Adjusted R-squared = -0.38
Fold 5 RMSE = 54458.17
Fold 5 Adjusted R-squared = -0.37
Mean RMSE = 55057.32 ± 1377.428
Mean Adjusted R-squared = -0.37



Ada Boost Regressor:

Fold 1 RMSE = 60757.47
Fold 1 Adjusted R-squared = -0.67
Fold 2 RMSE = 64286.54
Fold 2 Adjusted R-squared = -0.79
Fold 3 RMSE = 61495.67
Fold 3 Adjusted R-squared = -0.82
Fold 4 RMSE = 67153.44
Fold 4 Adjusted R

In [44]:
def objective(trial):
    params = {
        'loss': trial.suggest_categorical('loss', ['absolute_error', 'poisson', 'squared_error']),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 1.0, step = 0.2),
        'max_iter': trial.suggest_int('max_iter', 100, 1000, step = 50),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 30, 10000, step = 100),
        'max_depth': trial.suggest_int('max_depth', 30, 10000, step = 100),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 1000, step = 15),
        'l2_regularization': trial.suggest_float('l2_regularization', 0.01, 100, step = 0.05)
    }
    
    clf = HistGradientBoostingRegressor(**params, random_state = 123)
    scores = []
    
    for i, (train_index, val_index) in enumerate(cv_splits):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
        print(f'Fold {i + 1} RMSE = {rmse:.2f}')
        
        scores.append(rmse)
    
    if i == len(cv_splits) - 1:
        mean_score = np.mean(scores)
        
    return -mean_score        
         

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 100, show_progress_bar = True)

[I 2023-10-02 10:52:11,745] A new study created in memory with name: no-name-daea68c8-0dc4-4354-a673-20456a4bc09b


  0%|          | 0/100 [00:00<?, ?it/s]

Fold 1 RMSE = 56949.81
Fold 2 RMSE = 58608.09
Fold 3 RMSE = 55615.92
Fold 4 RMSE = 61353.19
Fold 5 RMSE = 58625.71
[I 2023-10-02 10:52:16,472] Trial 0 finished with value: -58230.54349692259 and parameters: {'loss': 'squared_error', 'learning_rate': 0.7000000000000001, 'max_iter': 150, 'max_leaf_nodes': 6930, 'max_depth': 9230, 'min_samples_leaf': 520, 'l2_regularization': 84.46000000000001}. Best is trial 0 with value: -58230.54349692259.
Fold 1 RMSE = 61884.76
Fold 2 RMSE = 63137.49
Fold 3 RMSE = 60310.89
Fold 4 RMSE = 65005.70
Fold 5 RMSE = 61891.09
[I 2023-10-02 10:52:31,830] Trial 1 finished with value: -62445.98422644699 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.9, 'max_iter': 500, 'max_leaf_nodes': 4430, 'max_depth': 5130, 'min_samples_leaf': 595, 'l2_regularization': 80.11000000000001}. Best is trial 0 with value: -58230.54349692259.
Fold 1 RMSE = 57039.15
Fold 2 RMSE = 58622.69
Fold 3 RMSE = 55629.55
Fold 4 RMSE = 61372.84
Fold 5 RMSE = 58569.12
[I 2023-10-

Fold 5 RMSE = 56592.56
[I 2023-10-02 11:03:35,961] Trial 19 finished with value: -55985.22021817191 and parameters: {'loss': 'squared_error', 'learning_rate': 0.30000000000000004, 'max_iter': 350, 'max_leaf_nodes': 1630, 'max_depth': 9930, 'min_samples_leaf': 280, 'l2_regularization': 70.66000000000001}. Best is trial 12 with value: -54945.33337431564.
Fold 1 RMSE = 58113.49
Fold 2 RMSE = 59153.21
Fold 3 RMSE = 56230.22
Fold 4 RMSE = 62520.28
Fold 5 RMSE = 58651.32
[I 2023-10-02 11:03:42,096] Trial 20 finished with value: -58933.705555200206 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.7000000000000001, 'max_iter': 150, 'max_leaf_nodes': 3830, 'max_depth': 6330, 'min_samples_leaf': 355, 'l2_regularization': 26.710000000000004}. Best is trial 12 with value: -54945.33337431564.
Fold 1 RMSE = 53870.39
Fold 2 RMSE = 55595.02
Fold 3 RMSE = 52998.60
Fold 4 RMSE = 58268.60
Fold 5 RMSE = 54767.72
[I 2023-10-02 11:04:50,895] Trial 21 finished with value: -55100.06428342842 and 

Fold 5 RMSE = 55731.31
[I 2023-10-02 11:24:03,093] Trial 38 finished with value: -55473.36574010029 and parameters: {'loss': 'poisson', 'learning_rate': 0.1, 'max_iter': 850, 'max_leaf_nodes': 5830, 'max_depth': 7830, 'min_samples_leaf': 55, 'l2_regularization': 54.81}. Best is trial 12 with value: -54945.33337431564.
Fold 1 RMSE = 58241.46
Fold 2 RMSE = 59249.75
Fold 3 RMSE = 56273.79
Fold 4 RMSE = 62620.95
Fold 5 RMSE = 58516.79
[I 2023-10-02 11:24:05,650] Trial 39 finished with value: -58980.549903215746 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.30000000000000004, 'max_iter': 100, 'max_leaf_nodes': 9930, 'max_depth': 9530, 'min_samples_leaf': 445, 'l2_regularization': 13.26}. Best is trial 12 with value: -54945.33337431564.
Fold 1 RMSE = 61884.76
Fold 2 RMSE = 63137.49
Fold 3 RMSE = 60311.59
Fold 4 RMSE = 65006.39
Fold 5 RMSE = 61891.09
[I 2023-10-02 11:24:12,943] Trial 40 finished with value: -62446.26158910524 and parameters: {'loss': 'absolute_error', 'learnin

Fold 1 RMSE = 53541.95
Fold 2 RMSE = 55551.84
Fold 3 RMSE = 52728.62
Fold 4 RMSE = 58348.39
Fold 5 RMSE = 55375.83
[I 2023-10-02 11:40:52,678] Trial 58 finished with value: -55109.3250121677 and parameters: {'loss': 'poisson', 'learning_rate': 0.1, 'max_iter': 100, 'max_leaf_nodes': 3430, 'max_depth': 5130, 'min_samples_leaf': 40, 'l2_regularization': 30.76}. Best is trial 12 with value: -54945.33337431564.
Fold 1 RMSE = 68626.10
Fold 2 RMSE = 70174.55
Fold 3 RMSE = 67150.80
Fold 4 RMSE = 70787.50
Fold 5 RMSE = 68038.38
[I 2023-10-02 11:40:55,361] Trial 59 finished with value: -68955.46825923855 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.1, 'max_iter': 350, 'max_leaf_nodes': 5230, 'max_depth': 6030, 'min_samples_leaf': 925, 'l2_regularization': 42.11}. Best is trial 12 with value: -54945.33337431564.
Fold 1 RMSE = 54123.99
Fold 2 RMSE = 56015.22
Fold 3 RMSE = 52926.02
Fold 4 RMSE = 58476.99
Fold 5 RMSE = 55088.98
[I 2023-10-02 11:42:14,605] Trial 60 finished with val

Fold 5 RMSE = 55714.53
[I 2023-10-02 11:57:07,644] Trial 77 finished with value: -55758.50820224164 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.1, 'max_iter': 200, 'max_leaf_nodes': 1830, 'max_depth': 6830, 'min_samples_leaf': 70, 'l2_regularization': 68.06}. Best is trial 74 with value: -54939.75672557547.
Fold 1 RMSE = 54437.49
Fold 2 RMSE = 56645.18
Fold 3 RMSE = 52573.76
Fold 4 RMSE = 58035.76
Fold 5 RMSE = 56511.83
[I 2023-10-02 11:57:14,466] Trial 78 finished with value: -55640.80409474698 and parameters: {'loss': 'squared_error', 'learning_rate': 0.1, 'max_iter': 150, 'max_leaf_nodes': 5930, 'max_depth': 8030, 'min_samples_leaf': 175, 'l2_regularization': 40.11}. Best is trial 74 with value: -54939.75672557547.
Fold 1 RMSE = 53575.99
Fold 2 RMSE = 55927.42
Fold 3 RMSE = 52895.10
Fold 4 RMSE = 58427.67
Fold 5 RMSE = 54881.64
[I 2023-10-02 11:57:50,650] Trial 79 finished with value: -55141.563393364544 and parameters: {'loss': 'absolute_error', 'learning_rate': 0

Fold 1 RMSE = 53934.62
Fold 2 RMSE = 55830.06
Fold 3 RMSE = 52688.80
Fold 4 RMSE = 58353.08
Fold 5 RMSE = 54961.47
[I 2023-10-02 12:24:44,436] Trial 97 finished with value: -55153.60501408621 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.1, 'max_iter': 550, 'max_leaf_nodes': 3130, 'max_depth': 4630, 'min_samples_leaf': 10, 'l2_regularization': 66.21000000000001}. Best is trial 84 with value: -54912.51009651328.
Fold 1 RMSE = 54178.39
Fold 2 RMSE = 56247.98
Fold 3 RMSE = 52451.79
Fold 4 RMSE = 57917.60
Fold 5 RMSE = 56058.48
[I 2023-10-02 12:25:17,244] Trial 98 finished with value: -55370.849158294324 and parameters: {'loss': 'squared_error', 'learning_rate': 0.1, 'max_iter': 650, 'max_leaf_nodes': 7230, 'max_depth': 5630, 'min_samples_leaf': 130, 'l2_regularization': 77.51}. Best is trial 84 with value: -54912.51009651328.
Fold 1 RMSE = 54842.79
Fold 2 RMSE = 56190.69
Fold 3 RMSE = 53127.29
Fold 4 RMSE = 58885.77
Fold 5 RMSE = 55867.04
[I 2023-10-02 12:26:33,741] Trial 

In [45]:
best_params = study.best_params
best_rmse_score = study.best_value


print(f'\nHistogram-based Gradient Boosting Regressor:\n')
print(f'\n Best RMSE score = {best_rmse_score} \n')
print(f'\n Best Params = {best_params} \n')


Histogram-based Gradient Boosting Regressor:


 Best RMSE score = -54912.51009651328 


 Best Params = {'loss': 'absolute_error', 'learning_rate': 0.1, 'max_iter': 700, 'max_leaf_nodes': 4130, 'max_depth': 5530, 'min_samples_leaf': 25, 'l2_regularization': 55.51} 



In [46]:
def objective2(trial):
    params = {
        'loss_function': trial.suggest_categorical('loss_function', ['MAE', 'RMSE', 'Poisson']),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, step=0.01),
        'iterations': trial.suggest_int('iterations', 100, 1000, step=50),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10, step=0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'random_strength': trial.suggest_float('random_strength', 0.1, 10, step=0.1),
        'eval_metric': 'RMSE'
    }
    
    clf = CatBoostRegressor(**params, random_state = 123, verbose = False)
    scores = []
    
    for i, (train_index, val_index) in enumerate(cv_splits):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
        print(f'Fold {i + 1} RMSE = {rmse:.2f}')
        
        scores.append(rmse)
    
    if i == len(cv_splits) - 1:
        mean_score = np.mean(scores)
        
    return -mean_score        
         

study2 = optuna.create_study(direction='maximize')
study2.optimize(objective2, n_trials = 100, show_progress_bar = True)

[I 2023-10-02 12:26:33,847] A new study created in memory with name: no-name-a4778fe5-af8d-4090-b043-7c837e2a3cb9


  0%|          | 0/100 [00:00<?, ?it/s]

Fold 1 RMSE = 53856.62
Fold 2 RMSE = 55035.99
Fold 3 RMSE = 52505.47
Fold 4 RMSE = 58467.68
Fold 5 RMSE = 56276.57
[I 2023-10-02 12:27:08,895] Trial 0 finished with value: -55228.46802843764 and parameters: {'loss_function': 'MAE', 'learning_rate': 0.2, 'iterations': 850, 'max_depth': 9, 'l2_leaf_reg': 6.6, 'min_data_in_leaf': 45, 'random_strength': 3.9000000000000004}. Best is trial 0 with value: -55228.46802843764.
Fold 1 RMSE = 55269.21
Fold 2 RMSE = 54244.09
Fold 3 RMSE = 52595.29
Fold 4 RMSE = 56612.17
Fold 5 RMSE = 54899.98
[I 2023-10-02 12:27:24,273] Trial 1 finished with value: -54724.14915474611 and parameters: {'loss_function': 'RMSE', 'learning_rate': 0.23, 'iterations': 550, 'max_depth': 7, 'l2_leaf_reg': 5.5, 'min_data_in_leaf': 45, 'random_strength': 3.7}. Best is trial 1 with value: -54724.14915474611.
Fold 1 RMSE = 157781.33
Fold 2 RMSE = 157536.06
Fold 3 RMSE = 160318.71
Fold 4 RMSE = 159887.76
Fold 5 RMSE = 156112.31
[I 2023-10-02 12:27:36,761] Trial 2 finished with v

Fold 1 RMSE = 52496.75
Fold 2 RMSE = 54800.20
Fold 3 RMSE = 52096.97
Fold 4 RMSE = 57219.22
Fold 5 RMSE = 53473.14
[I 2023-10-02 12:31:06,740] Trial 20 finished with value: -54017.25606386507 and parameters: {'loss_function': 'MAE', 'learning_rate': 0.12, 'iterations': 650, 'max_depth': 7, 'l2_leaf_reg': 6.1, 'min_data_in_leaf': 17, 'random_strength': 8.8}. Best is trial 7 with value: -53583.25017229925.
Fold 1 RMSE = 52318.12
Fold 2 RMSE = 54300.43
Fold 3 RMSE = 51760.64
Fold 4 RMSE = 56836.85
Fold 5 RMSE = 52996.38
[I 2023-10-02 12:31:12,221] Trial 21 finished with value: -53642.4851162027 and parameters: {'loss_function': 'MAE', 'learning_rate': 0.19, 'iterations': 200, 'max_depth': 6, 'l2_leaf_reg': 4.6, 'min_data_in_leaf': 1, 'random_strength': 8.4}. Best is trial 7 with value: -53583.25017229925.
Fold 1 RMSE = 52006.08
Fold 2 RMSE = 54692.69
Fold 3 RMSE = 51508.05
Fold 4 RMSE = 56725.09
Fold 5 RMSE = 53106.97
[I 2023-10-02 12:31:19,545] Trial 22 finished with value: -53607.775895

Fold 1 RMSE = 52285.27
Fold 2 RMSE = 53879.69
Fold 3 RMSE = 52135.09
Fold 4 RMSE = 56197.84
Fold 5 RMSE = 53497.28
[I 2023-10-02 12:33:09,838] Trial 40 finished with value: -53599.0347616154 and parameters: {'loss_function': 'RMSE', 'learning_rate': 0.09, 'iterations': 750, 'max_depth': 5, 'l2_leaf_reg': 8.0, 'min_data_in_leaf': 26, 'random_strength': 6.9}. Best is trial 39 with value: -53422.82276050739.
Fold 1 RMSE = 52303.86
Fold 2 RMSE = 54569.48
Fold 3 RMSE = 51481.55
Fold 4 RMSE = 56693.13
Fold 5 RMSE = 52874.99
[I 2023-10-02 12:33:18,866] Trial 41 finished with value: -53584.602783678994 and parameters: {'loss_function': 'MAE', 'learning_rate': 0.04, 'iterations': 600, 'max_depth': 5, 'l2_leaf_reg': 9.1, 'min_data_in_leaf': 36, 'random_strength': 5.9}. Best is trial 39 with value: -53422.82276050739.
Fold 1 RMSE = 51962.36
Fold 2 RMSE = 54552.09
Fold 3 RMSE = 51297.71
Fold 4 RMSE = 57004.05
Fold 5 RMSE = 52782.42
[I 2023-10-02 12:33:25,048] Trial 42 finished with value: -53519.7

Fold 1 RMSE = 157781.33
Fold 2 RMSE = 157536.06
Fold 3 RMSE = 160318.71
Fold 4 RMSE = 159887.76
Fold 5 RMSE = 156112.31
[I 2023-10-02 12:37:02,353] Trial 60 finished with value: -158327.23183437382 and parameters: {'loss_function': 'Poisson', 'learning_rate': 0.01, 'iterations': 950, 'max_depth': 8, 'l2_leaf_reg': 9.1, 'min_data_in_leaf': 27, 'random_strength': 2.4000000000000004}. Best is trial 45 with value: -53342.899005027546.
Fold 1 RMSE = 51790.26
Fold 2 RMSE = 54543.33
Fold 3 RMSE = 51100.44
Fold 4 RMSE = 56965.55
Fold 5 RMSE = 52548.00
[I 2023-10-02 12:37:13,863] Trial 61 finished with value: -53389.51777740347 and parameters: {'loss_function': 'MAE', 'learning_rate': 0.060000000000000005, 'iterations': 1000, 'max_depth': 4, 'l2_leaf_reg': 9.5, 'min_data_in_leaf': 24, 'random_strength': 0.1}. Best is trial 45 with value: -53342.899005027546.
Fold 1 RMSE = 51839.46
Fold 2 RMSE = 54496.04
Fold 3 RMSE = 51105.62
Fold 4 RMSE = 56862.88
Fold 5 RMSE = 52642.00
[I 2023-10-02 12:37:23,

Fold 1 RMSE = 52096.33
Fold 2 RMSE = 54794.23
Fold 3 RMSE = 51775.08
Fold 4 RMSE = 57469.48
Fold 5 RMSE = 52800.92
[I 2023-10-02 12:41:40,635] Trial 80 finished with value: -53787.206123280266 and parameters: {'loss_function': 'MAE', 'learning_rate': 0.13, 'iterations': 900, 'max_depth': 5, 'l2_leaf_reg': 7.6, 'min_data_in_leaf': 39, 'random_strength': 2.1}. Best is trial 66 with value: -53295.650887666896.
Fold 1 RMSE = 51738.05
Fold 2 RMSE = 54575.42
Fold 3 RMSE = 51148.15
Fold 4 RMSE = 56923.86
Fold 5 RMSE = 52569.35
[I 2023-10-02 12:41:53,001] Trial 81 finished with value: -53390.96468791816 and parameters: {'loss_function': 'MAE', 'learning_rate': 0.05, 'iterations': 950, 'max_depth': 4, 'l2_leaf_reg': 7.5, 'min_data_in_leaf': 43, 'random_strength': 0.7000000000000001}. Best is trial 66 with value: -53295.650887666896.
Fold 1 RMSE = 51721.53
Fold 2 RMSE = 54427.59
Fold 3 RMSE = 50997.39
Fold 4 RMSE = 56798.88
Fold 5 RMSE = 52640.66
[I 2023-10-02 12:42:04,790] Trial 82 finished wit

In [47]:
best_params2 = study2.best_params
best_rmse_score2 = study2.best_value


print(f'\nCatBoost Regressor:\n')
print(f'\n Best RMSE score = {best_rmse_score2} \n')
print(f'\n Best Params = {best_params2} \n')


CatBoost Regressor:


 Best RMSE score = -53295.650887666896 


 Best Params = {'loss_function': 'MAE', 'learning_rate': 0.05, 'iterations': 900, 'max_depth': 4, 'l2_leaf_reg': 8.3, 'min_data_in_leaf': 50, 'random_strength': 0.5} 



In [48]:
estimators = [
    ('CatBoost Regressor', CatBoostRegressor(**best_params2,random_state = 123, verbose = False)),
    ('Histogram-based Gradient Boosting', HistGradientBoostingRegressor(**best_params,random_state = 123))
]

In [49]:
# Creating meta model
meta_model = StackingRegressor(estimators = estimators,
                              n_jobs = 5)

In [50]:
print('\nMeta Model Cross-Validation:')
scores = []
r2_scores = []
    
print('\n')
print(f'\nStacking Regressor w/ CatBoost Regressor and Histogram-based Gradient Boosting Regressor :\n')
    
for i, (train_index, val_index) in enumerate(cv_splits):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
    meta_model.fit(X_train, y_train)
    y_pred = meta_model.predict(X_val)
        
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    r2 = r2_score(y_val, y_pred)
        
    n = X_val.shape[0]  
    p = X_val.shape[1]  
        
    adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1)) 
    
    print(f'Fold {i + 1} RMSE = {rmse:.2f}')
    print(f'Fold {i + 1} Adjusted R-squared = {adjusted_r2:.2f}')
        
    scores.append(rmse)
    r2_scores.append(adjusted_r2)
        
    print('===================================================')
    
    if i == len(cv_splits) - 1:
        mean_score = np.mean(scores)
        fold_std = np.std(scores)
        mean_adjusted_r2 = np.mean(r2_scores)  
        
        print(f'Mean RMSE = {mean_score:.2f} \u00B1 {fold_std:.3f}')
        print(f'Mean Adjusted R-squared = {mean_adjusted_r2:.2f}')


Meta Model Cross-Validation:



Stacking Regressor w/ CatBoost Regressor and Histogram-based Gradient Boosting Regressor :

Fold 1 RMSE = 51166.66
Fold 1 Adjusted R-squared = -0.19
Fold 2 RMSE = 54107.04
Fold 2 Adjusted R-squared = -0.27
Fold 3 RMSE = 50847.05
Fold 3 Adjusted R-squared = -0.25
Fold 4 RMSE = 56404.06
Fold 4 Adjusted R-squared = -0.36
Fold 5 RMSE = 52339.89
Fold 5 Adjusted R-squared = -0.26
Mean RMSE = 52972.94 ± 2061.145
Mean Adjusted R-squared = -0.27


In [51]:
def pred_vs_true_plot(y_true, y_pred):
    '''
    This function takes values for y_true and y_val, and plots a scatterplot along with a line of best fit
    '''

    slope, intercept = np.polyfit(y_true, y_pred, 1)
    fit_line = slope * y_true + intercept

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=y_true, y=y_pred, mode='markers', name='Data Points'))
    fig.add_trace(go.Scatter(x=y_true, y=fit_line, mode='lines', line=dict(color='red'), name='Fit-line'))
    fig.update_traces(marker=dict(size=10, color='blue'))
    fig.update_layout(
        title={'text': f'<b>True x Predicted <br> <sup>Scatterplot</sup></b>'},
        xaxis=dict(title='True Salaries'), 
        yaxis=dict(title='Predicted Salaries'),
        height=750,
        width=850,
        margin=dict(t=250, l=80),
        template='simple_white',
    )
    fig.show()


In [52]:
test # Visualizing test data

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
1482,MLOps Engineer,Full-Time,Mid,Intermediate,134000,United States Dollar,United States,134000,United States,Medium,2023
2738,Data Analyst,Full-Time,Senior,Expert,117000,United States Dollar,United States,117000,United States,Medium,2022
2743,Machine Learning Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2022
2424,Data Engineer,Full-Time,Senior,Expert,172200,United States Dollar,United States,172200,United States,Medium,2022
805,Machine Learning Engineer,Full-Time,Senior,Expert,180000,United States Dollar,United States,180000,United States,Medium,2023
...,...,...,...,...,...,...,...,...,...,...,...
1866,Research Scientist,Full-Time,Mid,Intermediate,23000,United States Dollar,India,23000,India,Large,2022
265,Data Analyst,Full-Time,Senior,Expert,70000,United States Dollar,United States,70000,United States,Medium,2023
2430,Head of Data,Full-Time,Executive,Director,160000,United States Dollar,United States,160000,United States,Medium,2022
1324,Analytics Engineer,Full-Time,Senior,Expert,143200,United States Dollar,United States,143200,United States,Medium,2023


In [53]:
# Saving salaries
true_salary = test['Salary in USD']
true_salary

1482    134000
2738    117000
2743    129300
2424    172200
805     180000
         ...  
1866     23000
265      70000
2430    160000
1324    143200
902     123906
Name: Salary in USD, Length: 660, dtype: int64

In [54]:
# Removing real labels
test = test.drop(['Salary in USD', 'Salary'], axis = 1)
# Encoding
test[['Experience Level', 'Expertise Level', 'Company Size']] = oe.transform(test[['Experience Level', 'Expertise Level', 'Company Size']])
test.head(5)

,Job Title,Employment Type,Experience Level,Expertise Level,Salary Currency,Company Location,Employee Residence,Company Size,Year
1482,MLOps Engineer,Full-Time,1.0,1.0,United States Dollar,United States,United States,1.0,2023
2738,Data Analyst,Full-Time,2.0,2.0,United States Dollar,United States,United States,1.0,2022
2743,Machine Learning Engineer,Full-Time,2.0,2.0,United States Dollar,United States,United States,1.0,2022
2424,Data Engineer,Full-Time,2.0,2.0,United States Dollar,United States,United States,1.0,2022
805,Machine Learning Engineer,Full-Time,2.0,2.0,United States Dollar,United States,United States,1.0,2023


In [55]:
cols = ['Job Title', 'Employment Type', 'Salary Currency', 'Company Location', 'Employee Residence']

# Encoding
encoded_features_test = pd.DataFrame(ohe.transform(test[cols]), 
                                     columns=ohe.get_feature_names_out(cols))
encoded_features_test.index = test.index
test = test.drop(cols, axis=1)
test = pd.concat([test, encoded_features_test], axis=1)
test

,Experience Level,Expertise Level,Company Size,Year,Job Title_AI Architect,Job Title_AI Developer,Job Title_AI Programmer,Job Title_AI Scientist,Job Title_Analytics Engineer,Job Title_Analytics Engineering Manager,...,Employee Residence_Sweden,Employee Residence_Switzerland,Employee Residence_Thailand,Employee Residence_Turkey,Employee Residence_Ukraine,Employee Residence_United Arab Emirates,Employee Residence_United Kingdom,Employee Residence_United States,Employee Residence_Uzbekistan,Employee Residence_Viet Nam
1482,1.0,1.0,1.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2738,2.0,2.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2743,2.0,2.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2424,2.0,2.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
805,2.0,2.0,1.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1866,1.0,1.0,2.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
265,2.0,2.0,1.0,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2430,3.0,3.0,1.0,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1324,2.0,2.0,1.0,2023,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [56]:
# Predicting with meta model
y_pred_meta = meta_model.predict(test)
y_pred_meta # Visualizing predictions

array([137574.83956883, 123267.20384562, 188937.11338715, 165407.72318969,
       196688.02152391,  97101.92377051, 193648.3035917 ,  59904.51467354,
        81696.80726459, 196688.02152391, 130994.72618658, 189366.35247797,
        30448.11858849, 170663.10633427,  81696.80726459, 177680.17955283,
       196428.73558964,  38867.56943688, 109464.84333169, 184246.71139365,
       194764.76296435, 144126.98808287, 173254.22481268, 173699.5628629 ,
       172453.10186921,  88159.10573259,  70987.79903613, 123267.20384562,
       167859.47720922, 173254.22481268, 165407.72318969, 129174.38133984,
       121005.03189123,  53106.35009392, 175908.23287518,  45737.60801686,
       157258.64624782, 187061.80405689, 152018.75898065, 196688.02152391,
       131162.6453947 , 173699.5628629 , 161999.33753194, 149235.49332679,
        47486.97890459,  96643.72308656,  92663.50556371, 100942.09313013,
       196428.73558964, 169978.75798072, 202468.56311386, 170663.10633427,
       137922.61664442, 1

In [57]:
predictions = pd.DataFrame({
    'id': test.index,
    'Real Salary': true_salary,
    'Predicted Salary': y_pred_meta
})
predictions

,id,Real Salary,Predicted Salary
1482,1482,134000,137574.839569
2738,2738,117000,123267.203846
2743,2743,129300,188937.113387
2424,2424,172200,165407.723190
805,805,180000,196688.021524
...,...,...,...
1866,1866,23000,87067.064098
265,265,70000,129174.381340
2430,2430,160000,194565.584978
1324,1324,143200,165043.249464


In [58]:
print('\nTest Results with Meta Model: \n')
rmse = mean_squared_error(predictions['Real Salary'], predictions['Predicted Salary'], squared=False)
print(f'RMSE = {rmse:.2f}')

r2 = r2_score(predictions['Real Salary'], predictions['Predicted Salary'])
n = X_val.shape[0]  
p = X_val.shape[1]  
adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
print(f'Adjusted R-Squared = {adjusted_r2:.2f}')

pred_vs_true_plot(true_salary, y_pred_meta)


Test Results with Meta Model: 

RMSE = 51170.60
Adjusted R-Squared = -0.13
